# MobileNet для различных предметных областей

1. Изучить архитектуру MobileNet и провести частичное обучение, экспорт и импорт модели, затем дообучение.
2. Перейти от задачи классификации изображения к задаче локализации объекта на изображении при помощи якорей.
3. Оценить качество предсказания по коэффициенту сходства.

## Подключение библиотек

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import keras
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, BatchNormalization, Dropout
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.regularizers import l2
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# import warnings
# warnings.filterwarnings('ignore')

## Используемые функции

In [16]:
filesDir = "../data/clouds/train_images_small/"
batch_size = 10
image_x = 525
image_y = 350
image_ch = 3
mask_x = 5
mask_y = 5

def mask_rate(a, x, y):
    b = a // 1400 + 0.0
    return np.round(x*(b*x // 2100) + y*(a%1400) // 1400).astype("uint32")

def calc_mask(px, x=image_x, y=image_y):
    p = np.array([int(n) for n in px.split(" ")]).reshape(-1, 2)
    mask = np.zeros(y*x, dtype="uint8")
    for i, m in p:
        mask[mask_rate(i, x, y)-1:mask_rate(m + i, x, y)] = 1
    return mask.reshape(y, x).transpose()

def calc_dice(x):
    dice = 0
    px = x["EncodedPixels"]
    if px != px and x["target"] == 0:
        dice = 1
    elif px == px and x["target"] == 1:
        mask = calc_mask(px).flatten()
        target = np.kron(np.array(x["MaskPixels"].split(" ")).reshape(mask_x, mask_y).astype("i1"),
                         np.ones((image_y // mask_y, image_x // mask_x), dtype="i1")).transpose().flatten()
        dice = 2*np.sum(target[mask==1]) / (np.sum(target) + np.sum(mask))
    return dice

def load_y(df):
    y = [[0]]*len(df)
    for i, ep in enumerate(df["EncodedPixels"]):
        if ep == ep:
            y[i] = calc_mask(ep, mask_x, mask_y).transpose().flatten()
        else:
            y[i] = np.zeros(mask_x * mask_y, dtype="i1")
    return np.array(y).reshape(len(df), mask_y, mask_x, 1)

def load_x(df):
    x = [[]]*len(df)
    for j, file in enumerate(df["Image"]):
        img = image.load_img(os.path.join(filesDir, file), target_size=(image_y, image_x))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        x[j] = preprocess_input(img)
    return np.array(x).reshape(len(df), image_y, image_x, image_ch)

def load_data(df, batch_size):
    while True:
        batch_start = 0
        batch_end = batch_size
        while batch_start < len(df):
            limit = min(batch_end, len(df))
            yield (load_x(df[batch_start:limit]),
                   load_y(df[batch_start:limit]))
            batch_start += batch_size
            batch_end += batch_size
            
def draw_prediction(prediction):
    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot(1, 1, 1)
    ax.hist(prediction[0])
    ax.set_title("Fish")
    plt.show()

## Загрузка данных

In [3]:
data = pd.read_csv("../data/clouds/train.csv.gz")

In [4]:
data["Image"] = data["Image_Label"].str.split("_").str[0]
data["Label"] = data["Image_Label"].str.split("_").str[1]
data.drop(labels=["Image_Label"], axis=1, inplace=True)
data_fish = data[data["Label"] == "Fish"]
print(data_fish.head())

                                        EncodedPixels        Image Label
0   264918 937 266318 937 267718 937 269118 937 27...  0011165.jpg  Fish
4   233813 878 235213 878 236613 878 238010 881 23...  002be4f.jpg  Fish
8   3510 690 4910 690 6310 690 7710 690 9110 690 1...  0031ae9.jpg  Fish
12                                                NaN  0035239.jpg  Fish
16  2367966 18 2367985 2 2367993 8 2368002 62 2369...  003994e.jpg  Fish


## Разделение данных

In [5]:
train, test = train_test_split(data_fish, test_size=0.2)
del data
print(train.head())

                                           EncodedPixels        Image Label
3204                                                 NaN  2404bd2.jpg  Fish
10848  32201 1367 33601 1367 35001 1367 36401 1367 37...  7b7d5ef.jpg  Fish
20560  440152 657 441552 657 442952 657 444352 657 44...  ecefc50.jpg  Fish
19472                                                NaN  e0d46d0.jpg  Fish
18240  1109543 533 1110943 533 1112343 533 1113743 53...  d2e7aa7.jpg  Fish


## MobileNet v2

In [8]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_y, image_x, image_ch))
base_model_output = base_model.predict_generator(load_data(train, 1), steps=len(train), verbose=1)
top_model = Sequential([
    Flatten(input_shape=base_model_output.shape[1:]),
    BatchNormalization(),
    Dropout(0.5),
    Activation("softmax"),
    Dense(mask_x * mask_y)
])
top_model.compile(optimizer=optimizers.Nadam(lr=0.05), loss="mean_absolute_error")

In [ ]:
top_model.fit(base_model_output,
              load_y(train).reshape(len(train), -1), epochs=100,
              callbacks=[EarlyStopping(monitor="loss", min_delta=0.0001, patience=5, verbose=1, mode="auto"),
                         ModelCheckpoint("mobilenet.clouds.h5", mode="auto", monitor="val_loss", verbose=1)])

## Продолжение обучения

In [ ]:
del top_model
top_model = keras.models.load_model("mobilenet.clouds.h5")

In [ ]:
top_model.fit(base_model_output,
              load_y(train).reshape(len(train), -1), epochs=100,
              callbacks=[EarlyStopping(monitor="loss", min_delta=0.0001, patience=5, verbose=1, mode="auto"),
                         ModelCheckpoint("mobilenet.clouds.h5", mode="auto", monitor="val_loss", verbose=1)])

In [ ]:
model = Model(inputs=base_model.input,
              outputs=top_model(base_model.output))
model.compile(optimizer="adam", loss="mean_absolute_error")
model.summary()

## Построение предсказания

In [ ]:
prediction = model.predict_generator(load_data(test, 1), steps=len(test), verbose=1)

In [ ]:
prediction = (prediction > 0.5).astype("i1")

In [ ]:
print(prediction[0:10])
print(load_y(test.head(10)).reshape(10, -1))

In [ ]:
target = []
masks = []
for i, vals in enumerate(prediction):
    if vals.sum() > 4:
        targ = 1
    else:
        targ = 0
    target.append(targ)
    masks.append(np.array2string(vals.flatten().astype("int8"), separator=" ")[1:-1])
test["MaskPixels"] = masks
test["target"] = target
print(test[test["target"] > 0][["EncodedPixels", "MaskPixels"]].head(20))

## Расчёт точности предсказания

In [ ]:
dice = test.apply(calc_dice, axis=1, result_type="expand")
print("Keras, MobileNet:", round(dice.mean(), 3))